In [17]:
import os
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from sentinelhub import SHConfig
import getpass
import tqdm as notebook_tqdm

In [19]:
config = SHConfig()
config.sh_client_id = getpass.getpass(os.getenv('CLIENT_ID'))
config.sh_client_secret = getpass.getpass(os.getenv('CLIENT_SECRET'))
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.save("cdse")

In [20]:
def getauth_token():
    # Create a session
    client = BackendApplicationClient(client_id=config.sh_client_id)
    oauth = OAuth2Session(client=client)
    # Get token for the session
    token = oauth.fetch_token(
        token_url="https://identity.cloudferro.com/auth/realms/CDSE/protocol/openid-connect/token",
        client_id=config.sh_client_id,
        client_secret=config.sh_client_secret,
    )
    # All requests using this session will have an access token automatically added

    return oauth

In [21]:
def evaluatePixel(samples):
    #detection of vegetation
    NDVI_RedEdge = (samples.B08 - samples.B05)/(samples.B08 + samples.B05)
    threshold_vegetation = 0.45
    Vegetation = NDVI_RedEdge > threshold_vegetation

    #ceramic rooftop detection
    RATIO_Red = samples.B04/[samples.B01+samples.B02+samples.B03+samples.B04+samples.B05+samples.B06+samples.B07]
    NDBI = (samples.B11 - samples.B08)/(samples.B11 + samples.B08)
    threshold_rooftop = 0.14
    Rooftop = (RATIO_Red > threshold_rooftop) and (NDBI > threshold_rooftop)

    #water detection
    NDWI = (samples.B03 - samples.B08)/(samples.B03 + samples.B08)
    threshold_water = 0.2
    Water = NDWI > threshold_water

    #gain to obtain smooth visualization
    gain = 0.7
    return [gain*Rooftop, gain*Vegetation, gain*Water]

In [30]:
def get_request():
    evalscript="""NDVI_RedEdge = (B08 - B05)/(B08 + B05)
threshold_vegetation = 0.45
Vegetation = NDVI_RedEdge > threshold_vegetation

// ceramic rooftop detection
RATIO_Red = B04/[B01+B02+B03+B04+B05+B06+B07]
NDBI = (B11 - B08)/(B11 + B08)
threshold_rooftop = 0.14
Rooftop = (RATIO_Red > threshold_rooftop) && (NDBI > threshold_rooftop)

// water detection
NDWI = (B03 - B08)/(B03 + B08)
threshold_water = 0.2
Water = NDWI > threshold_water

// gain to obtain smooth visualization
gain = 0.7
return [gain*Rooftop, gain*Vegetation, gain*Water]"""
    request = {
        "input": {
            "bounds": {
                "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
                "bbox": [21.331921, 41.896182, 21.531921, 42.096182],
            },
            "data": [
                {
                    "dataFilter": {
                        "timeRange": {
                            "from": "2025-04-16T00:00:00Z",
                            "to": "2025-05-16T23:59:59Z"
                        }
                    },
                    "type": "sentinel-2-l2a",
                }
            ],
        },
        "output": {
            "width": 512.792,
            "height": 687.801,
            "responses": [
                {
                    "identifier": "default",
                    "format": {"type": "image/tiff"},
                }
            ],
        },
        "evalscript": evalscript,
    }

    url = "https://sh.dataspace.copernicus.eu/api/v1/process"
    oauth=getauth_token()
    response = oauth.post(url, json=request, headers={"Accept": "image/tiff"})

    if response.status_code in (200,):
        with open(f"urban_skopje.tiff", "wb") as tarfile:
            tarfile.write(response.content)


    return response

In [31]:
print(get_request().content)

b'MM\x00*\x00\x00\x00\x08\x00\x10\x01\x00\x00\x03\x00\x00\x00\x01\x02\x00\x00\x00\x01\x01\x00\x03\x00\x00\x00\x01\x02\xaf\x00\x00\x01\x02\x00\x03\x00\x00\x00\x03\x00\x00\x00\xd0\x01\x03\x00\x03\x00\x00\x00\x01\x80\xb2\x00\x00\x01\x06\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\x01\x11\x00\x04\x00\x00\x00V\x00\x00\x00\xd8\x01\x15\x00\x03\x00\x00\x00\x01\x00\x03\x00\x00\x01\x16\x00\x03\x00\x00\x00\x01\x00\x08\x00\x00\x01\x17\x00\x04\x00\x00\x00V\x00\x00\x020\x01\x1a\x00\x05\x00\x00\x00\x01\x00\x00\x03\x88\x01\x1b\x00\x05\x00\x00\x00\x01\x00\x00\x03\x90\x01(\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\x83\x0e\x00\x0c\x00\x00\x00\x03\x00\x00\x03\x98\x84\x82\x00\x0c\x00\x00\x00\x06\x00\x00\x03\xb0\x87\xaf\x00\x03\x00\x00\x00\x1c\x00\x00\x03\xe0\x87\xb1\x00\x02\x00\x00\x00\x0f\x00\x00\x04\x18\x00\x00\x00\x00\x00\x00\x00\x08\x00\x08\x00\x08\x00\x00\x00\x00\x04\'\x00\x00\x07\x1f\x00\x00\n\x14\x00\x00\r\x05\x00\x00\x10\x14\x00\x00\x13Z\x00\x00\x16j\x00\x00\x19j\x00\x00\x1c\x8c\x00\x00\x1f\xa9\x00\x00"\

In [46]:
def get_request_green():
    evalscript="""var ndvi = (B08-B04)/(B08+B04);

// Threshold for vegetation
var veg_th = 0.4;

// Simple RGB
var R = 2.5*B04;
var G = 2.5*B03;
var B = 2.5*B02;

// Transform to Black and White
var Y = 0.2*R + 0.7*G + 0.1*B;
var pixel = [Y, Y, Y];

// Change vegetation color
if(ndvi >= veg_th)
  pixel = [0.1*Y, 1.8*Y, 0.1*Y];

return pixel;"""
    request = {
        "input": {
            "bounds": {
                "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
                "bbox": [21.331921, 41.896182, 21.531921, 42.096182],
            },
            "data": [
                {
                    "dataFilter": {
                        "timeRange": {
                            "from": "2025-04-16T00:00:00Z",
                            "to": "2025-05-16T23:59:59Z"
                        }
                    },
                    "type": "sentinel-2-l2a",
                }
            ],
        },
        "output": {
            "width": 512.792,
            "height": 687.801,
            "responses": [
                {
                    "identifier": "default",
                    "format": {"type": "image/tiff"},
                }
            ],
        },
        "evalscript": evalscript,
    }

    url = "https://sh.dataspace.copernicus.eu/api/v1/process"
    oauth=getauth_token()
    response = oauth.post(url, json=request, headers={"Accept": "image/tiff"})

    if response.status_code in (200,):
        with open(f"D:\\Hristijan Gajdov\\Projects\\Python\\CassiniHackathon\\green_aktebe.tiff", "wb") as tarfile:
            tarfile.write(response.content)


    return response

In [36]:
print(get_request_green())

<Response [200]>


In [56]:
def get_request_ozone():
    evalscript = """const band = "O3";
var minVal = 0.0;
var maxVal = 0.36;

function setup() {
  return {
    input: [band, "dataMask"],
    output: {
      bands: 4,
    },
  };
}

var viz = ColorRampVisualizer.createBlueRed(minVal, maxVal);

function evaluatePixel(samples) {
  let ret = viz.process(samples[band]);
  ret.push(samples.dataMask);
  return ret;
}

"""
    request = {
        "input": {
            "bounds": {
                "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
                "bbox": [21.331921, 41.896182, 21.531921, 42.096182],
            },
            "data": [
                {
                    "dataFilter": {
                        "timeRange": {
                            "from": "2025-04-16T00:00:00Z",
                            "to": "2025-05-16T23:59:59Z"
                        }
                    },
                    "type": "sentinel-5p-l2",
                }
            ],
        },
        "output": {
            "width": 512.792,
            "height": 687.801,
            "responses": [
                {
                    "identifier": "default",
                    "format": {"type": "image/tiff"},
                }
            ],
        },
        "evalscript": evalscript,
    }

    url = "https://sh.dataspace.copernicus.eu/api/v1/process"
    oauth = getauth_token()
    response = oauth.post(url, json=request, headers={"Accept": "image/tiff"})

    if response.status_code in (200,):
        with open(f"ozone_skopje.tiff", "wb") as tarfile:
            tarfile.write(response.content)

    return response

In [57]:
print(get_request_ozone())

<Response [200]>
